<a href="https://colab.research.google.com/github/invoker-qqwrv/Beida_tensorflow/blob/main/4_6acc%26loss_visualization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 训练集loss：loss
# 测试集loss：val_loss
# 训练集准确率：sparse_categorical_accuracy
# 测试集准确率:val_sparse_categorical_accuracy
# 接下来代码实现训练集和测试集acc&loss曲线数据集的显示

#入门级的八股一定能用这套代码实现

In [ ]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt

np.set_printoptions(threshold=np.inf)

fashion = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fashion.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['sparse_categorical_accuracy'])

checkpoint_save_path = "./checkpoint/fashion.ckpt"
if os.path.exists(checkpoint_save_path + '.index'):
    print('-------------load the model-----------------')
    model.load_weights(checkpoint_save_path)

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_save_path,
                        save_weights_only=True,
                        save_best_only=True)

history = model.fit(x_train, y_train, batch_size=32, epochs=5, validation_data=(x_test, y_test), validation_freq=1,
                    callbacks=[cp_callback])
model.summary()

print(model.trainable_variables)
file = open('./weights.txt', 'w')
for v in model.trainable_variables:
    file.write(str(v.name) + '\n')
    file.write(str(v.shape) + '\n')
    file.write(str(v.numpy()) + '\n')
file.close()

###############################################show###############################################

# 显示训练集和验证集的acc和loss曲线
acc = history.history['sparse_categorical_accuracy']#用history.history函数提取model.fit函数中保存的训练集准确率
val_acc = history.history['val_sparse_categorical_accuracy']
loss = history.history['loss']#训练集损失函数数值
val_loss = history.history['val_loss']#测试集损失函数数值

plt.subplot(1, 2, 1)#分成一行两列，画出第一列
plt.plot(acc, label='Training Accuracy')#图标题
plt.plot(val_acc, label='Validation Accuracy')
plt.title('Training and Validation Accuracy')#tuli 
plt.legend()

plt.subplot(1, 2, 2)#以下代码画出第二列
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()


In [11]:
from PIL import Image
import numpy as np
import tensorflow as tf

model_save_path = './checkpoint/fashion.ckpt'
#复现网络
model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')])
#加载参数    
model.load_weights(model_save_path)
#询问要执行多少次图像识别任务
preNum = int(input("input the number of test pictures:"))
#读入待识别图片
for i in range(preNum):
    image_path = input("the path of test picture:")
    img = Image.open(image_path)
    #模型用的图片是28 28 的灰度图，所以要resize这个
    img = img.resize((28, 28), Image.ANTIALIAS)
    img_arr = np.array(img.convert('L'))
# 应用程序的输入图片是白底黑字，但我们的输入图片是黑底白字灰度图，所以要用255减去当前像素值，相当于颜色取反。这个也就是预处理
# 有很多图片的预处理。比如把小于某值的定为0，大于该值的定为1.

    img_arr = 255 - img_arr
                
    img_arr = img_arr / 255.0
    print("img_arr:",img_arr.shape)
    x_predict = img_arr[tf.newaxis, ...]
    print("x_predict:",x_predict.shape)
    result = model.predict(x_predict)
    
    pred = tf.argmax(result, axis=1)
    
    print('\n')
    tf.print(pred)

input the number of test pictures:10
the path of test picture:0_t.jpeg


FileNotFoundError: ignored

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
